# Случайные леса
__Суммарное количество баллов: 10__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[HSE][ML][HW09] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy

In [2]:
def gini(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return np.sum(proba * (1 - proba))
    
def entropy(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return -np.sum(proba * np.log2(proba))

def gain(left_y, right_y, criterion):
    y = np.concatenate((left_y, right_y))
    return criterion(y) - (criterion(left_y) * len(left_y) + criterion(right_y) * len(right_y)) / len(y)

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [3]:
from collections import Counter
from typing import Callable, Union, NoReturn, Optional, Dict, Any, List

class DecisionTreeLeaf:
    """

    Attributes
    ----------
    y : Тип метки (напр., int или str)
        Метка класса, который встречается чаще всего среди элементов листа дерева
    """
    def __init__(self, sub_y):
        self.y = Counter(sub_y).most_common(1)[0][0]
        

class DecisionTreeNode:
    """

    Attributes
    ----------
    split_dim : int
        Измерение, по которому разбиваем выборку.
    split_value : float
        Значение, по которому разбираем выборку.
    left : Union[DecisionTreeNode, DecisionTreeLeaf]
        Поддерево, отвечающее за случай x[split_dim] < split_value.
    right : Union[DecisionTreeNode, DecisionTreeLeaf]
        Поддерево, отвечающее за случай x[split_dim] >= split_value. 
    """
    def __init__(self, split_dim: int, split_value: float, 
                 left: Union['DecisionTreeNode', DecisionTreeLeaf], 
                 right: Union['DecisionTreeNode', DecisionTreeLeaf]):
        self.split_dim = split_dim
        self.split_value = split_value
        self.left = left
        self.right = right

In [4]:
def find_best_split(feature_vector, target_vector, min_split, criterion=gini):
    
    s_ind = np.argsort(np.array(feature_vector))
    x = np.array(feature_vector)[s_ind]
    y = np.array(target_vector)[s_ind]


    n = x.shape[0]

    thresholds = np.unique(x)
    if thresholds.shape[0] == 1:
        return (None, None)
    thresholds = (thresholds[:-1] + thresholds[1:]) / 2
    left, right = None, None
    ind = 0
    best_gain, best_threshold = None, None
    tmp = []
    for threshold in thresholds:
        while x[ind] < threshold and ind < n:
            ind += 1
        if ind < min_split:
            continue
        if n - ind < min_split:
            break
        left = y[:ind]
        right = y[ind:]
        ig = gain(left, right, criterion)
        tmp.append(ig)
        if best_gain is None or ig > best_gain:
            best_gain = ig
            best_threshold = threshold
    
    return best_threshold, best_gain

In [5]:
class DecisionTree:
    def __init__(self, X, y, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto"):
        n = X.shape[0]
        indexes = np.random.randint(0, n, n)
        self.X = X[indexes]
        self.y = y[indexes]
        out_of_bag_ind = [i not in set(indexes) for i in range(n)]
        self.out_of_bag_x = X[out_of_bag_ind]
        self.out_of_bag_y = y[out_of_bag_ind]
        self.criterion = gini
        if criterion == "entropy":
            self.criterion = entropy
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = int(np.sqrt(X.shape[1]))
        if max_features != "auto":
            self.max_features = max_features
        self.root = self._fit_node(self.X, self.y, 0)
    
    def out_of_bag(self):
        return [self.out_of_bag_x, self.out_of_bag_y]
            
    def _fit_node(self, sub_X, sub_y, depth):
        if np.all(sub_y == sub_y[0]):
            node = DecisionTreeLeaf(sub_y)
            return node
        
        if self.max_depth is not None and depth >= self.max_depth:
            node = DecisionTreeLeaf(sub_y)
            return node
        feature_best, threshold_best, gain_best, split = None, None, None, None
        feature_indexes = np.random.randint(0, sub_X.shape[1], self.max_features)
        for feature in feature_indexes:
            feature_vector = sub_X[:, feature]
            
            threshold, ig = find_best_split(feature_vector, sub_y, self.min_samples_leaf, self.criterion)
            if threshold is not None and (gain_best is None or ig > gain_best):
                feature_best = feature
                gain_best = ig
                split = feature_vector < threshold
                threshold_best = threshold
        if feature_best is None:
            node = DecisionTreeLeaf(sub_y)
            return node
        left = self._fit_node(sub_X[split], sub_y[split], depth + 1)
        right = self._fit_node(sub_X[~split], sub_y[~split], depth + 1)
        node = DecisionTreeNode(feature_best, threshold_best, left, right)
        return node

    def _predict_node(self, x, node):
        if isinstance(node, DecisionTreeLeaf):
            return node.y
        
        f = node.split_dim

        if x[f] < node.split_value:
            return self._predict_node(x, node.left)
        else:
            return self._predict_node(x, node.right)

        
    def predict(self, X):
        predicted = []
        for x in X:
            predicted.append(self._predict_node(x, self.root))
        return predicted

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [6]:
from tqdm.notebook import tqdm

class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.n_estimators = n_estimators
        
    def out_of_bag(self):
        oob = [tree.out_of_bag() for tree in self.trees]
        return oob
    
    def fit(self, X, y):
        self.trees = [DecisionTree(X, y, self.criterion, self.max_depth, self.min_samples_leaf, self.max_features) for _ in tqdm(range(self.n_estimators))]
    
    def predict(self, X):
        cur = []
        for tree in self.trees:
            cur.append(tree.predict(X))
        result = []
        cur = np.array(cur)
        #print(cur)
        for i in range(X.shape[0]):
            ideas = cur[:, i]
            values, counts = np.unique(ideas, return_counts=True)
            ind = np.argmax(counts)
            result.append(values[ind])
        return result

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [7]:
def feature_importance(rfc, sample=1):
    out_of_bag = rfc.out_of_bag()
    res = 0
    for i in range(len(out_of_bag)):
        pred_y = rfc.predict(out_of_bag[i][0])
        res += np.mean(pred_y == out_of_bag[i][1])
    result = []
    for i in range(len(out_of_bag)):
        x = np.array(out_of_bag[i][0])[::sample].copy()
        for j in tqdm(range(x.shape[1])):
            x = np.array(out_of_bag[i][0])[::sample].copy()
            if i == 0:
                result.append(res)
            x_j = x[:, j]
            random.shuffle(x_j)
            x[:, j] = x_j
            pred_y = rfc.predict(x)
            err_oob_j = np.mean(pred_y == out_of_bag[i][1][::sample])
            result[j] -= err_oob_j
    return result

def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [8]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=20)
rfc.fit(X, y)
pred = rfc.predict(X)
#print(pred, y)
print("Accuracy:", np.mean(pred == y))
print("Importance:", feature_importance(rfc))


Accuracy: 1.0



Importance: [0.0, 0.0, 4.841411667282861, 3.783495319471448, 7.781898521362044, 0.0]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [9]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [10]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [11]:
rfc = RandomForestClassifier(n_estimators=10)

rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc, sample=10), features, 20)):
    print(str(i+1) + ".", name)


Accuracy: 0.725094577553594
Most important features:



1. mudakoff
2. ovsyanochan
3. 4ch
4. styd.pozor
5. rapnewrap
6. rhymes
7. dayvinchik
8. iwantyou
9. pravdashowtop
10. bot_maxim
11. tumblr_vacuum
12. borsch
13. fuck_humor
14. xfilm
15. exclusive_muzic
16. top_screens
17. tnt
18. soverwenstvo.decora
19. pixel_stickers
20. thesmolny


#### Пол

In [12]:
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc, sample=10), features, 20)):
    print(str(i+1) + ".", name)


Accuracy: 0.832282471626734
Most important features:



1. 40kg
2. mudakoff
3. modnailru
4. cook_good
5. be.beauty
6. sh.cook
7. zerofat
8. 9o_6o_9o
9. girlmeme
10. 4ch
11. be.women
12. igm
13. femalemem
14. rapnewrap
15. beauty
16. reflexia_our_feelings
17. pixel_stickers
18. thesmolny
19. h.made
20. decadence


### CatBoost
В качестве аьтернативы попробуем CatBoost. 

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [13]:
from catboost import CatBoostClassifier

In [14]:
X, y = synthetic_dataset(1000)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True)

print(y_train)

classifier = CatBoostClassifier(loss_function='MultiClass')
classifier.fit(X_train, y_train)

[1 1 0 1 0 0 1 1 0 1 0 0 0 2 2 1 2 0 1 0 0 0 1 2 0 2 0 2 1 0 1 0 1 1 2 1 2
 1 2 1 1 0 0 2 1 1 2 0 0 2 2 2 0 0 2 2 2 0 1 0 1 2 0 2 1 1 1 1 1 0 0 1 2 1
 0 1 2 2 0 0 2 1 2 1 0 2 2 1 2 2 2 0 2 0 2 2 1 0 2 2 0 1 0 1 1 1 0 2 0 2 2
 0 2 0 0 2 2 2 1 1 1 1 0 1 2 2 1 1 2 2 0 1 2 2 1 0 2 1 1 2 2 1 0 0 1 2 1 0
 0 0 0 2 1 0 2 1 1 1 1 1 2 0 1 1 2 2 0 1 1 1 0 0 2 1 0 0 2 2 2 0 2 0 0 0 2
 0 0 2 2 1 1 0 1 2 1 2 1 2 2 0 1 0 2 1 0 1 2 1 1 1 0 1 2 2 2 1 2 0 0 1 0 0
 0 1 1 0 2 2 1 1 1 1 0 0 1 0 2 1 1 0 0 1 0 1 0 2 0 2 0 2 1 2 1 1 1 0 0 2 0
 2 0 1 0 0 2 0 0 0 1 0 1 2 1 0 0 0 0 2 0 1 0 2 0 1 0 2 1 1 2 2 2 0 2 2 2 1
 1 1 0 1 0 2 0 0 2 1 1 2 1 1 2 0 0 0 1 2 1 1 0 2 1 2 2 0 2 0 0 1 2 2 0 0 2
 1 2 2 2 2 0 1 0 0 1 2 2 2 0 1 2 1 0 0 2 0 0 1 2 1 1 1 1 0 1 2 2 0 0 0 2 1
 0 1 0 2 2 0 1 1 1 0 2 1 0 0 1 0 2 1 1 2 1 1 2 1 2 0 2 0 1 1 0 1 2 0 1 1 0
 0 2 0 1 1 2 1 0 2 1 0 1 1 1 2 0 2 0 2 1 0 2 0 0 0 0 0 1 1 1 0 0 2 2 1 0 0
 1 1 2 2 2 2 0 1 2 0 2 1 2 0 1 2 0 0 2 2 2 0 0 1 2 0 1 0 2 1 1 1 1 0 1 2 1
 1 2 2 1 0 0 0 1 0 1 1 2 

157:	learn: 0.0030033	total: 132ms	remaining: 705ms
158:	learn: 0.0029888	total: 133ms	remaining: 702ms
159:	learn: 0.0029745	total: 133ms	remaining: 700ms
160:	learn: 0.0029602	total: 134ms	remaining: 698ms
161:	learn: 0.0029462	total: 134ms	remaining: 696ms
162:	learn: 0.0029322	total: 135ms	remaining: 693ms
163:	learn: 0.0029184	total: 136ms	remaining: 691ms
164:	learn: 0.0029047	total: 136ms	remaining: 689ms
165:	learn: 0.0028911	total: 137ms	remaining: 687ms
166:	learn: 0.0028660	total: 137ms	remaining: 684ms
167:	learn: 0.0028527	total: 138ms	remaining: 682ms
168:	learn: 0.0028396	total: 138ms	remaining: 680ms
169:	learn: 0.0028266	total: 139ms	remaining: 678ms
170:	learn: 0.0028138	total: 139ms	remaining: 676ms
171:	learn: 0.0028010	total: 140ms	remaining: 674ms
172:	learn: 0.0027884	total: 141ms	remaining: 672ms
173:	learn: 0.0027649	total: 141ms	remaining: 670ms
174:	learn: 0.0027526	total: 142ms	remaining: 667ms
175:	learn: 0.0027403	total: 142ms	remaining: 666ms
176:	learn: 

423:	learn: 0.0012947	total: 305ms	remaining: 414ms
424:	learn: 0.0012919	total: 306ms	remaining: 414ms
425:	learn: 0.0012892	total: 307ms	remaining: 414ms
426:	learn: 0.0012864	total: 309ms	remaining: 414ms
427:	learn: 0.0012837	total: 310ms	remaining: 414ms
428:	learn: 0.0012809	total: 310ms	remaining: 413ms
429:	learn: 0.0012782	total: 311ms	remaining: 413ms
430:	learn: 0.0012755	total: 312ms	remaining: 412ms
431:	learn: 0.0012728	total: 314ms	remaining: 412ms
432:	learn: 0.0012702	total: 315ms	remaining: 412ms
433:	learn: 0.0012675	total: 316ms	remaining: 412ms
434:	learn: 0.0012648	total: 317ms	remaining: 412ms
435:	learn: 0.0012622	total: 318ms	remaining: 411ms
436:	learn: 0.0012595	total: 319ms	remaining: 411ms
437:	learn: 0.0012569	total: 320ms	remaining: 411ms
438:	learn: 0.0012543	total: 321ms	remaining: 410ms
439:	learn: 0.0012517	total: 322ms	remaining: 410ms
440:	learn: 0.0012491	total: 323ms	remaining: 410ms
441:	learn: 0.0012465	total: 324ms	remaining: 409ms
442:	learn: 

668:	learn: 0.0008472	total: 481ms	remaining: 238ms
669:	learn: 0.0008460	total: 481ms	remaining: 237ms
670:	learn: 0.0008448	total: 482ms	remaining: 236ms
671:	learn: 0.0008436	total: 482ms	remaining: 235ms
672:	learn: 0.0008425	total: 483ms	remaining: 235ms
673:	learn: 0.0008413	total: 484ms	remaining: 234ms
674:	learn: 0.0008401	total: 484ms	remaining: 233ms
675:	learn: 0.0008389	total: 485ms	remaining: 232ms
676:	learn: 0.0008377	total: 485ms	remaining: 231ms
677:	learn: 0.0008366	total: 486ms	remaining: 231ms
678:	learn: 0.0008354	total: 486ms	remaining: 230ms
679:	learn: 0.0008342	total: 487ms	remaining: 229ms
680:	learn: 0.0008331	total: 487ms	remaining: 228ms
681:	learn: 0.0008319	total: 488ms	remaining: 227ms
682:	learn: 0.0008308	total: 488ms	remaining: 227ms
683:	learn: 0.0008296	total: 489ms	remaining: 226ms
684:	learn: 0.0008285	total: 489ms	remaining: 225ms
685:	learn: 0.0008273	total: 490ms	remaining: 224ms
686:	learn: 0.0008262	total: 490ms	remaining: 223ms
687:	learn: 

966:	learn: 0.0005954	total: 649ms	remaining: 22.2ms
967:	learn: 0.0005948	total: 650ms	remaining: 21.5ms
968:	learn: 0.0005942	total: 650ms	remaining: 20.8ms
969:	learn: 0.0005936	total: 651ms	remaining: 20.1ms
970:	learn: 0.0005930	total: 652ms	remaining: 19.5ms
971:	learn: 0.0005924	total: 652ms	remaining: 18.8ms
972:	learn: 0.0005919	total: 653ms	remaining: 18.1ms
973:	learn: 0.0005913	total: 653ms	remaining: 17.4ms
974:	learn: 0.0005907	total: 654ms	remaining: 16.8ms
975:	learn: 0.0005901	total: 655ms	remaining: 16.1ms
976:	learn: 0.0005895	total: 655ms	remaining: 15.4ms
977:	learn: 0.0005889	total: 656ms	remaining: 14.8ms
978:	learn: 0.0005883	total: 657ms	remaining: 14.1ms
979:	learn: 0.0005878	total: 658ms	remaining: 13.4ms
980:	learn: 0.0005872	total: 658ms	remaining: 12.8ms
981:	learn: 0.0005866	total: 659ms	remaining: 12.1ms
982:	learn: 0.0005860	total: 660ms	remaining: 11.4ms
983:	learn: 0.0005855	total: 660ms	remaining: 10.7ms
984:	learn: 0.0005849	total: 661ms	remaining: 

In [15]:
print("Accuracy:", np.mean(np.array(classifier.predict(X_test)).flatten() == y_test))
print("Importance:", classifier.get_feature_importance())

Accuracy: 1.0
Importance: [9.44661843e-03 1.68034240e-02 2.72552177e+01 2.75333597e+01
 4.51769608e+01 8.21179760e-03]


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [16]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [17]:
classifier = CatBoostClassifier(loss_function='MultiClass', iterations=10000)
classifier.fit(X_train, y_age_train)

Learning rate set to 0.012498
0:	learn: 1.0942894	total: 6.76ms	remaining: 1m 7s
1:	learn: 1.0901327	total: 11.9ms	remaining: 59.7s
2:	learn: 1.0856341	total: 16.9ms	remaining: 56.4s
3:	learn: 1.0813931	total: 21.7ms	remaining: 54.1s
4:	learn: 1.0772628	total: 26.5ms	remaining: 52.9s
5:	learn: 1.0732020	total: 31.6ms	remaining: 52.6s
6:	learn: 1.0694819	total: 36.8ms	remaining: 52.6s
7:	learn: 1.0652500	total: 42.3ms	remaining: 52.8s
8:	learn: 1.0613542	total: 48.1ms	remaining: 53.3s
9:	learn: 1.0575405	total: 53.1ms	remaining: 53s
10:	learn: 1.0539239	total: 57.9ms	remaining: 52.6s
11:	learn: 1.0503748	total: 62.8ms	remaining: 52.2s
12:	learn: 1.0470542	total: 68ms	remaining: 52.2s
13:	learn: 1.0439633	total: 72.8ms	remaining: 51.9s
14:	learn: 1.0406732	total: 77.8ms	remaining: 51.8s
15:	learn: 1.0373528	total: 82.9ms	remaining: 51.7s
16:	learn: 1.0342141	total: 88.3ms	remaining: 51.9s
17:	learn: 1.0310374	total: 93.9ms	remaining: 52.1s
18:	learn: 1.0276896	total: 99.1ms	remaining: 52

176:	learn: 0.7933549	total: 963ms	remaining: 53.4s
177:	learn: 0.7926054	total: 969ms	remaining: 53.5s
178:	learn: 0.7918597	total: 975ms	remaining: 53.5s
179:	learn: 0.7911447	total: 981ms	remaining: 53.5s
180:	learn: 0.7904383	total: 986ms	remaining: 53.5s
181:	learn: 0.7897253	total: 992ms	remaining: 53.5s
182:	learn: 0.7889616	total: 997ms	remaining: 53.5s
183:	learn: 0.7882392	total: 1s	remaining: 53.4s
184:	learn: 0.7874936	total: 1.01s	remaining: 53.4s
185:	learn: 0.7867395	total: 1.01s	remaining: 53.4s
186:	learn: 0.7860063	total: 1.02s	remaining: 53.3s
187:	learn: 0.7852980	total: 1.02s	remaining: 53.3s
188:	learn: 0.7845412	total: 1.03s	remaining: 53.3s
189:	learn: 0.7838135	total: 1.03s	remaining: 53.3s
190:	learn: 0.7831219	total: 1.04s	remaining: 53.3s
191:	learn: 0.7824065	total: 1.04s	remaining: 53.2s
192:	learn: 0.7816646	total: 1.05s	remaining: 53.2s
193:	learn: 0.7810345	total: 1.05s	remaining: 53.2s
194:	learn: 0.7803689	total: 1.06s	remaining: 53.2s
195:	learn: 0.7

362:	learn: 0.6997177	total: 1.94s	remaining: 51.4s
363:	learn: 0.6993884	total: 1.94s	remaining: 51.4s
364:	learn: 0.6990483	total: 1.95s	remaining: 51.4s
365:	learn: 0.6987375	total: 1.95s	remaining: 51.4s
366:	learn: 0.6983723	total: 1.96s	remaining: 51.4s
367:	learn: 0.6979859	total: 1.96s	remaining: 51.3s
368:	learn: 0.6976147	total: 1.97s	remaining: 51.3s
369:	learn: 0.6973064	total: 1.97s	remaining: 51.3s
370:	learn: 0.6969793	total: 1.98s	remaining: 51.3s
371:	learn: 0.6966332	total: 1.98s	remaining: 51.3s
372:	learn: 0.6963482	total: 1.99s	remaining: 51.3s
373:	learn: 0.6960477	total: 1.99s	remaining: 51.3s
374:	learn: 0.6957085	total: 2s	remaining: 51.3s
375:	learn: 0.6953688	total: 2s	remaining: 51.2s
376:	learn: 0.6949956	total: 2.01s	remaining: 51.2s
377:	learn: 0.6946584	total: 2.01s	remaining: 51.2s
378:	learn: 0.6943624	total: 2.02s	remaining: 51.2s
379:	learn: 0.6940413	total: 2.02s	remaining: 51.2s
380:	learn: 0.6937205	total: 2.03s	remaining: 51.2s
381:	learn: 0.6934

549:	learn: 0.6491077	total: 2.9s	remaining: 49.8s
550:	learn: 0.6489026	total: 2.9s	remaining: 49.8s
551:	learn: 0.6486805	total: 2.91s	remaining: 49.8s
552:	learn: 0.6484547	total: 2.92s	remaining: 49.8s
553:	learn: 0.6482731	total: 2.92s	remaining: 49.8s
554:	learn: 0.6480771	total: 2.93s	remaining: 49.8s
555:	learn: 0.6478473	total: 2.93s	remaining: 49.8s
556:	learn: 0.6476745	total: 2.94s	remaining: 49.8s
557:	learn: 0.6474827	total: 2.94s	remaining: 49.8s
558:	learn: 0.6472867	total: 2.95s	remaining: 49.8s
559:	learn: 0.6470373	total: 2.96s	remaining: 49.8s
560:	learn: 0.6468329	total: 2.96s	remaining: 49.8s
561:	learn: 0.6466267	total: 2.97s	remaining: 49.9s
562:	learn: 0.6464117	total: 2.98s	remaining: 49.9s
563:	learn: 0.6462260	total: 2.98s	remaining: 49.9s
564:	learn: 0.6459840	total: 2.99s	remaining: 49.9s
565:	learn: 0.6457748	total: 3s	remaining: 50s
566:	learn: 0.6455991	total: 3s	remaining: 50s
567:	learn: 0.6453596	total: 3.01s	remaining: 50s
568:	learn: 0.6451670	tota

722:	learn: 0.6175893	total: 3.87s	remaining: 49.7s
723:	learn: 0.6173999	total: 3.88s	remaining: 49.7s
724:	learn: 0.6172401	total: 3.88s	remaining: 49.7s
725:	learn: 0.6171127	total: 3.89s	remaining: 49.7s
726:	learn: 0.6169598	total: 3.89s	remaining: 49.7s
727:	learn: 0.6167785	total: 3.9s	remaining: 49.7s
728:	learn: 0.6166086	total: 3.9s	remaining: 49.6s
729:	learn: 0.6164698	total: 3.91s	remaining: 49.6s
730:	learn: 0.6163094	total: 3.91s	remaining: 49.6s
731:	learn: 0.6161300	total: 3.92s	remaining: 49.6s
732:	learn: 0.6159727	total: 3.92s	remaining: 49.6s
733:	learn: 0.6158288	total: 3.93s	remaining: 49.6s
734:	learn: 0.6156791	total: 3.93s	remaining: 49.6s
735:	learn: 0.6155343	total: 3.94s	remaining: 49.6s
736:	learn: 0.6153603	total: 3.94s	remaining: 49.6s
737:	learn: 0.6151804	total: 3.95s	remaining: 49.5s
738:	learn: 0.6150538	total: 3.95s	remaining: 49.5s
739:	learn: 0.6148795	total: 3.96s	remaining: 49.5s
740:	learn: 0.6146719	total: 3.96s	remaining: 49.5s
741:	learn: 0.

...

9277:	learn: 0.2913934	total: 1m 6s	remaining: 5.14s
9278:	learn: 0.2913679	total: 1m 6s	remaining: 5.13s
9279:	learn: 0.2913511	total: 1m 6s	remaining: 5.12s
9280:	learn: 0.2913325	total: 1m 6s	remaining: 5.12s
9281:	learn: 0.2913054	total: 1m 6s	remaining: 5.11s
9282:	learn: 0.2912918	total: 1m 6s	remaining: 5.1s
9283:	learn: 0.2912813	total: 1m 6s	remaining: 5.1s
9284:	learn: 0.2912659	total: 1m 6s	remaining: 5.09s
9285:	learn: 0.2912533	total: 1m 6s	remaining: 5.08s
9286:	learn: 0.2912377	total: 1m 6s	remaining: 5.07s
9287:	learn: 0.2912201	total: 1m 6s	remaining: 5.07s
9288:	learn: 0.2912029	total: 1m 6s	remaining: 5.06s
9289:	learn: 0.2911888	total: 1m 6s	remaining: 5.05s
9290:	learn: 0.2911752	total: 1m 6s	remaining: 5.04s
9291:	learn: 0.2911651	total: 1m 6s	remaining: 5.04s
9292:	learn: 0.2911496	total: 1m 6s	remaining: 5.03s
9293:	learn: 0.2911338	total: 1m 6s	remaining: 5.02s
9294:	learn: 0.2911256	total: 1m 6s	remaining: 5.02s
9295:	learn: 0.2911130	total: 1m 6s	remaining: 5

9457:	learn: 0.2884685	total: 1m 7s	remaining: 3.85s
9458:	learn: 0.2884573	total: 1m 7s	remaining: 3.85s
9459:	learn: 0.2884487	total: 1m 7s	remaining: 3.84s
9460:	learn: 0.2884258	total: 1m 7s	remaining: 3.83s
9461:	learn: 0.2883940	total: 1m 7s	remaining: 3.82s
9462:	learn: 0.2883738	total: 1m 7s	remaining: 3.82s
9463:	learn: 0.2883591	total: 1m 7s	remaining: 3.81s
9464:	learn: 0.2883456	total: 1m 7s	remaining: 3.8s
9465:	learn: 0.2883220	total: 1m 7s	remaining: 3.79s
9466:	learn: 0.2883111	total: 1m 7s	remaining: 3.79s
9467:	learn: 0.2882960	total: 1m 7s	remaining: 3.78s
9468:	learn: 0.2882792	total: 1m 7s	remaining: 3.77s
9469:	learn: 0.2882628	total: 1m 7s	remaining: 3.77s
9470:	learn: 0.2882528	total: 1m 7s	remaining: 3.76s
9471:	learn: 0.2882399	total: 1m 7s	remaining: 3.75s
9472:	learn: 0.2882332	total: 1m 7s	remaining: 3.75s
9473:	learn: 0.2882217	total: 1m 7s	remaining: 3.74s
9474:	learn: 0.2882031	total: 1m 7s	remaining: 3.73s
9475:	learn: 0.2881878	total: 1m 7s	remaining: 

9613:	learn: 0.2861013	total: 1m 8s	remaining: 2.74s
9614:	learn: 0.2860941	total: 1m 8s	remaining: 2.73s
9615:	learn: 0.2860776	total: 1m 8s	remaining: 2.72s
9616:	learn: 0.2860626	total: 1m 8s	remaining: 2.72s
9617:	learn: 0.2860451	total: 1m 8s	remaining: 2.71s
9618:	learn: 0.2860338	total: 1m 8s	remaining: 2.7s
9619:	learn: 0.2860156	total: 1m 8s	remaining: 2.7s
9620:	learn: 0.2860049	total: 1m 8s	remaining: 2.69s
9621:	learn: 0.2859874	total: 1m 8s	remaining: 2.68s
9622:	learn: 0.2859698	total: 1m 8s	remaining: 2.67s
9623:	learn: 0.2859499	total: 1m 8s	remaining: 2.67s
9624:	learn: 0.2859380	total: 1m 8s	remaining: 2.66s
9625:	learn: 0.2859226	total: 1m 8s	remaining: 2.65s
9626:	learn: 0.2859045	total: 1m 8s	remaining: 2.65s
9627:	learn: 0.2858889	total: 1m 8s	remaining: 2.64s
9628:	learn: 0.2858757	total: 1m 8s	remaining: 2.63s
9629:	learn: 0.2858658	total: 1m 8s	remaining: 2.63s
9630:	learn: 0.2858581	total: 1m 8s	remaining: 2.62s
9631:	learn: 0.2858397	total: 1m 8s	remaining: 2

9770:	learn: 0.2836506	total: 1m 9s	remaining: 1.63s
9771:	learn: 0.2836381	total: 1m 9s	remaining: 1.62s
9772:	learn: 0.2836255	total: 1m 9s	remaining: 1.61s
9773:	learn: 0.2836106	total: 1m 9s	remaining: 1.6s
9774:	learn: 0.2835995	total: 1m 9s	remaining: 1.6s
9775:	learn: 0.2835811	total: 1m 9s	remaining: 1.59s
9776:	learn: 0.2835674	total: 1m 9s	remaining: 1.58s
9777:	learn: 0.2835540	total: 1m 9s	remaining: 1.58s
9778:	learn: 0.2835370	total: 1m 9s	remaining: 1.57s
9779:	learn: 0.2835235	total: 1m 9s	remaining: 1.56s
9780:	learn: 0.2835074	total: 1m 9s	remaining: 1.56s
9781:	learn: 0.2834914	total: 1m 9s	remaining: 1.55s
9782:	learn: 0.2834781	total: 1m 9s	remaining: 1.54s
9783:	learn: 0.2834679	total: 1m 9s	remaining: 1.53s
9784:	learn: 0.2834609	total: 1m 9s	remaining: 1.53s
9785:	learn: 0.2834434	total: 1m 9s	remaining: 1.52s
9786:	learn: 0.2834218	total: 1m 9s	remaining: 1.51s
9787:	learn: 0.2834133	total: 1m 9s	remaining: 1.51s
9788:	learn: 0.2834012	total: 1m 9s	remaining: 1

9929:	learn: 0.2812662	total: 1m 10s	remaining: 496ms
9930:	learn: 0.2812551	total: 1m 10s	remaining: 489ms
9931:	learn: 0.2812473	total: 1m 10s	remaining: 482ms
9932:	learn: 0.2812306	total: 1m 10s	remaining: 475ms
9933:	learn: 0.2812135	total: 1m 10s	remaining: 468ms
9934:	learn: 0.2812017	total: 1m 10s	remaining: 461ms
9935:	learn: 0.2811870	total: 1m 10s	remaining: 454ms
9936:	learn: 0.2811691	total: 1m 10s	remaining: 446ms
9937:	learn: 0.2811509	total: 1m 10s	remaining: 439ms
9938:	learn: 0.2811387	total: 1m 10s	remaining: 432ms
9939:	learn: 0.2811220	total: 1m 10s	remaining: 425ms
9940:	learn: 0.2811062	total: 1m 10s	remaining: 418ms
9941:	learn: 0.2810909	total: 1m 10s	remaining: 411ms
9942:	learn: 0.2810763	total: 1m 10s	remaining: 404ms
9943:	learn: 0.2810581	total: 1m 10s	remaining: 397ms
9944:	learn: 0.2810484	total: 1m 10s	remaining: 390ms
9945:	learn: 0.2810307	total: 1m 10s	remaining: 383ms
9946:	learn: 0.2810104	total: 1m 10s	remaining: 376ms
9947:	learn: 0.2809935	total

In [18]:
print("Accuracy train:", np.mean(np.array(classifier.predict(X_train)).flatten() == np.array(y_age_train)))
print("Accuracy test:", np.mean(np.array(classifier.predict(X_test)).flatten() == np.array(y_age_test)))
print("Most important features:")
for i, name in enumerate(most_important_features(classifier.get_feature_importance(), features, 10)):
    print(str(i+1) + ".", name)

Accuracy train: 0.9199270789510587
Accuracy test: 0.7515762925598991
Most important features:
1. mudakoff
2. ovsyanochan
3. 4ch
4. styd.pozor
5. rhymes
6. dayvinchik
7. rapnewrap
8. leprum
9. xfilm
10. kino_mania


#### Пол

In [19]:
classifier = CatBoostClassifier(loss_function='MultiClass', iterations=3000)
classifier.fit(X_train, y_sex_train)

Learning rate set to 0.034567
0:	learn: 0.6844070	total: 3.69ms	remaining: 11.1s
1:	learn: 0.6769910	total: 7.34ms	remaining: 11s
2:	learn: 0.6701782	total: 10.9ms	remaining: 10.9s
3:	learn: 0.6631726	total: 14.3ms	remaining: 10.7s
4:	learn: 0.6557125	total: 17.7ms	remaining: 10.6s
5:	learn: 0.6494118	total: 21ms	remaining: 10.5s
6:	learn: 0.6430414	total: 24.5ms	remaining: 10.5s
7:	learn: 0.6369969	total: 28.1ms	remaining: 10.5s
8:	learn: 0.6306008	total: 31.4ms	remaining: 10.4s
9:	learn: 0.6246211	total: 34.9ms	remaining: 10.4s
10:	learn: 0.6189268	total: 38.4ms	remaining: 10.4s
11:	learn: 0.6141664	total: 41.7ms	remaining: 10.4s
12:	learn: 0.6090451	total: 45.1ms	remaining: 10.4s
13:	learn: 0.6043878	total: 48.6ms	remaining: 10.4s
14:	learn: 0.5993424	total: 52ms	remaining: 10.3s
15:	learn: 0.5952925	total: 55.4ms	remaining: 10.3s
16:	learn: 0.5906608	total: 58.7ms	remaining: 10.3s
17:	learn: 0.5861347	total: 62.1ms	remaining: 10.3s
18:	learn: 0.5820712	total: 65.4ms	remaining: 10.3

203:	learn: 0.3596962	total: 769ms	remaining: 10.5s
204:	learn: 0.3593689	total: 773ms	remaining: 10.5s
205:	learn: 0.3590104	total: 776ms	remaining: 10.5s
206:	learn: 0.3586411	total: 780ms	remaining: 10.5s
207:	learn: 0.3582752	total: 783ms	remaining: 10.5s
208:	learn: 0.3579014	total: 787ms	remaining: 10.5s
209:	learn: 0.3574956	total: 791ms	remaining: 10.5s
210:	learn: 0.3571291	total: 795ms	remaining: 10.5s
211:	learn: 0.3567481	total: 798ms	remaining: 10.5s
212:	learn: 0.3564118	total: 802ms	remaining: 10.5s
213:	learn: 0.3558752	total: 805ms	remaining: 10.5s
214:	learn: 0.3555361	total: 809ms	remaining: 10.5s
215:	learn: 0.3552031	total: 812ms	remaining: 10.5s
216:	learn: 0.3548203	total: 816ms	remaining: 10.5s
217:	learn: 0.3544344	total: 819ms	remaining: 10.5s
218:	learn: 0.3540603	total: 823ms	remaining: 10.4s
219:	learn: 0.3537706	total: 827ms	remaining: 10.4s
220:	learn: 0.3534071	total: 830ms	remaining: 10.4s
221:	learn: 0.3531607	total: 833ms	remaining: 10.4s
222:	learn: 

409:	learn: 0.3067105	total: 1.54s	remaining: 9.71s
410:	learn: 0.3065275	total: 1.54s	remaining: 9.7s
411:	learn: 0.3063748	total: 1.54s	remaining: 9.7s
412:	learn: 0.3060890	total: 1.55s	remaining: 9.69s
413:	learn: 0.3059530	total: 1.55s	remaining: 9.69s
414:	learn: 0.3056889	total: 1.55s	remaining: 9.69s
415:	learn: 0.3055625	total: 1.56s	remaining: 9.68s
416:	learn: 0.3053477	total: 1.56s	remaining: 9.68s
417:	learn: 0.3051553	total: 1.56s	remaining: 9.67s
418:	learn: 0.3049647	total: 1.57s	remaining: 9.67s
419:	learn: 0.3048408	total: 1.57s	remaining: 9.66s
420:	learn: 0.3046349	total: 1.58s	remaining: 9.66s
421:	learn: 0.3044912	total: 1.58s	remaining: 9.65s
422:	learn: 0.3043584	total: 1.58s	remaining: 9.65s
423:	learn: 0.3041916	total: 1.59s	remaining: 9.64s
424:	learn: 0.3040266	total: 1.59s	remaining: 9.64s
425:	learn: 0.3038494	total: 1.59s	remaining: 9.63s
426:	learn: 0.3037162	total: 1.6s	remaining: 9.63s
427:	learn: 0.3035393	total: 1.6s	remaining: 9.62s
428:	learn: 0.30

608:	learn: 0.2785549	total: 2.29s	remaining: 8.98s
609:	learn: 0.2784267	total: 2.29s	remaining: 8.98s
610:	learn: 0.2783303	total: 2.3s	remaining: 8.98s
611:	learn: 0.2781716	total: 2.3s	remaining: 8.98s
612:	learn: 0.2779621	total: 2.31s	remaining: 8.98s
613:	learn: 0.2778728	total: 2.31s	remaining: 8.97s
614:	learn: 0.2777704	total: 2.31s	remaining: 8.97s
615:	learn: 0.2776424	total: 2.32s	remaining: 8.96s
616:	learn: 0.2775195	total: 2.32s	remaining: 8.96s
617:	learn: 0.2774170	total: 2.32s	remaining: 8.96s
618:	learn: 0.2773301	total: 2.33s	remaining: 8.95s
619:	learn: 0.2772123	total: 2.33s	remaining: 8.95s
620:	learn: 0.2771378	total: 2.33s	remaining: 8.95s
621:	learn: 0.2770009	total: 2.34s	remaining: 8.95s
622:	learn: 0.2769017	total: 2.34s	remaining: 8.94s
623:	learn: 0.2768254	total: 2.35s	remaining: 8.94s
624:	learn: 0.2767099	total: 2.35s	remaining: 8.94s
625:	learn: 0.2766172	total: 2.35s	remaining: 8.93s
626:	learn: 0.2764606	total: 2.36s	remaining: 8.93s
627:	learn: 0.

813:	learn: 0.2577297	total: 3.06s	remaining: 8.21s
814:	learn: 0.2576478	total: 3.06s	remaining: 8.21s
815:	learn: 0.2575619	total: 3.06s	remaining: 8.2s
816:	learn: 0.2574359	total: 3.07s	remaining: 8.2s
817:	learn: 0.2573567	total: 3.07s	remaining: 8.19s
818:	learn: 0.2572800	total: 3.08s	remaining: 8.19s
819:	learn: 0.2571765	total: 3.08s	remaining: 8.18s
820:	learn: 0.2571087	total: 3.08s	remaining: 8.18s
821:	learn: 0.2570256	total: 3.08s	remaining: 8.18s
822:	learn: 0.2569275	total: 3.09s	remaining: 8.17s
823:	learn: 0.2568212	total: 3.09s	remaining: 8.16s
824:	learn: 0.2566974	total: 3.1s	remaining: 8.16s
825:	learn: 0.2566270	total: 3.1s	remaining: 8.16s
826:	learn: 0.2565432	total: 3.1s	remaining: 8.15s
827:	learn: 0.2564529	total: 3.11s	remaining: 8.15s
828:	learn: 0.2563837	total: 3.11s	remaining: 8.14s
829:	learn: 0.2563272	total: 3.11s	remaining: 8.14s
830:	learn: 0.2562391	total: 3.12s	remaining: 8.13s
831:	learn: 0.2561499	total: 3.12s	remaining: 8.13s
832:	learn: 0.256

...

2216:	learn: 0.1835042	total: 9.5s	remaining: 3.35s
2217:	learn: 0.1834690	total: 9.51s	remaining: 3.35s
2218:	learn: 0.1834444	total: 9.52s	remaining: 3.35s
2219:	learn: 0.1834031	total: 9.52s	remaining: 3.35s
2220:	learn: 0.1833598	total: 9.53s	remaining: 3.34s
2221:	learn: 0.1833418	total: 9.54s	remaining: 3.34s
2222:	learn: 0.1833197	total: 9.55s	remaining: 3.34s
2223:	learn: 0.1832828	total: 9.55s	remaining: 3.33s
2224:	learn: 0.1832416	total: 9.56s	remaining: 3.33s
2225:	learn: 0.1832025	total: 9.57s	remaining: 3.33s
2226:	learn: 0.1831683	total: 9.58s	remaining: 3.33s
2227:	learn: 0.1831276	total: 9.59s	remaining: 3.32s
2228:	learn: 0.1830779	total: 9.6s	remaining: 3.32s
2229:	learn: 0.1830380	total: 9.61s	remaining: 3.32s
2230:	learn: 0.1830179	total: 9.61s	remaining: 3.31s
2231:	learn: 0.1829720	total: 9.62s	remaining: 3.31s
2232:	learn: 0.1829379	total: 9.63s	remaining: 3.31s
2233:	learn: 0.1829149	total: 9.63s	remaining: 3.3s
2234:	learn: 0.1828994	total: 9.64s	remaining: 3.

2377:	learn: 0.1780976	total: 10.7s	remaining: 2.79s
2378:	learn: 0.1780672	total: 10.7s	remaining: 2.79s
2379:	learn: 0.1780344	total: 10.7s	remaining: 2.79s
2380:	learn: 0.1780161	total: 10.7s	remaining: 2.78s
2381:	learn: 0.1779615	total: 10.7s	remaining: 2.78s
2382:	learn: 0.1779347	total: 10.7s	remaining: 2.77s
2383:	learn: 0.1779002	total: 10.7s	remaining: 2.77s
2384:	learn: 0.1778685	total: 10.7s	remaining: 2.77s
2385:	learn: 0.1778198	total: 10.7s	remaining: 2.76s
2386:	learn: 0.1777764	total: 10.7s	remaining: 2.76s
2387:	learn: 0.1777365	total: 10.7s	remaining: 2.75s
2388:	learn: 0.1777014	total: 10.8s	remaining: 2.75s
2389:	learn: 0.1776652	total: 10.8s	remaining: 2.75s
2390:	learn: 0.1776162	total: 10.8s	remaining: 2.74s
2391:	learn: 0.1775552	total: 10.8s	remaining: 2.74s
2392:	learn: 0.1775288	total: 10.8s	remaining: 2.73s
2393:	learn: 0.1775064	total: 10.8s	remaining: 2.73s
2394:	learn: 0.1774586	total: 10.8s	remaining: 2.72s
2395:	learn: 0.1774301	total: 10.8s	remaining:

2560:	learn: 0.1722226	total: 11.5s	remaining: 1.96s
2561:	learn: 0.1721781	total: 11.5s	remaining: 1.96s
2562:	learn: 0.1721461	total: 11.5s	remaining: 1.96s
2563:	learn: 0.1721170	total: 11.5s	remaining: 1.95s
2564:	learn: 0.1720919	total: 11.5s	remaining: 1.95s
2565:	learn: 0.1720541	total: 11.5s	remaining: 1.94s
2566:	learn: 0.1720305	total: 11.5s	remaining: 1.94s
2567:	learn: 0.1719918	total: 11.5s	remaining: 1.93s
2568:	learn: 0.1719630	total: 11.5s	remaining: 1.93s
2569:	learn: 0.1719432	total: 11.5s	remaining: 1.92s
2570:	learn: 0.1719059	total: 11.5s	remaining: 1.92s
2571:	learn: 0.1718766	total: 11.5s	remaining: 1.92s
2572:	learn: 0.1718430	total: 11.5s	remaining: 1.91s
2573:	learn: 0.1718198	total: 11.5s	remaining: 1.91s
2574:	learn: 0.1717786	total: 11.5s	remaining: 1.9s
2575:	learn: 0.1717460	total: 11.5s	remaining: 1.9s
2576:	learn: 0.1717221	total: 11.5s	remaining: 1.89s
2577:	learn: 0.1716875	total: 11.5s	remaining: 1.89s
2578:	learn: 0.1716590	total: 11.5s	remaining: 1

2721:	learn: 0.1674034	total: 12.4s	remaining: 1.27s
2722:	learn: 0.1673813	total: 12.5s	remaining: 1.27s
2723:	learn: 0.1673552	total: 12.5s	remaining: 1.26s
2724:	learn: 0.1673252	total: 12.5s	remaining: 1.26s
2725:	learn: 0.1673115	total: 12.5s	remaining: 1.26s
2726:	learn: 0.1672928	total: 12.5s	remaining: 1.25s
2727:	learn: 0.1672532	total: 12.5s	remaining: 1.25s
2728:	learn: 0.1672360	total: 12.6s	remaining: 1.25s
2729:	learn: 0.1672174	total: 12.6s	remaining: 1.24s
2730:	learn: 0.1671862	total: 12.6s	remaining: 1.24s
2731:	learn: 0.1671615	total: 12.6s	remaining: 1.23s
2732:	learn: 0.1671258	total: 12.6s	remaining: 1.23s
2733:	learn: 0.1671080	total: 12.6s	remaining: 1.23s
2734:	learn: 0.1670944	total: 12.6s	remaining: 1.22s
2735:	learn: 0.1670811	total: 12.6s	remaining: 1.22s
2736:	learn: 0.1670584	total: 12.6s	remaining: 1.21s
2737:	learn: 0.1670220	total: 12.6s	remaining: 1.21s
2738:	learn: 0.1669861	total: 12.6s	remaining: 1.2s
2739:	learn: 0.1669585	total: 12.6s	remaining: 

2898:	learn: 0.1624712	total: 13.6s	remaining: 475ms
2899:	learn: 0.1624508	total: 13.6s	remaining: 470ms
2900:	learn: 0.1624145	total: 13.6s	remaining: 466ms
2901:	learn: 0.1623934	total: 13.6s	remaining: 461ms
2902:	learn: 0.1623570	total: 13.7s	remaining: 456ms
2903:	learn: 0.1623307	total: 13.7s	remaining: 452ms
2904:	learn: 0.1623022	total: 13.7s	remaining: 447ms
2905:	learn: 0.1622676	total: 13.7s	remaining: 442ms
2906:	learn: 0.1622361	total: 13.7s	remaining: 438ms
2907:	learn: 0.1622219	total: 13.7s	remaining: 433ms
2908:	learn: 0.1622113	total: 13.7s	remaining: 428ms
2909:	learn: 0.1621774	total: 13.7s	remaining: 424ms
2910:	learn: 0.1621552	total: 13.7s	remaining: 419ms
2911:	learn: 0.1621442	total: 13.7s	remaining: 415ms
2912:	learn: 0.1621170	total: 13.7s	remaining: 410ms
2913:	learn: 0.1620951	total: 13.7s	remaining: 406ms
2914:	learn: 0.1620680	total: 13.8s	remaining: 401ms
2915:	learn: 0.1620475	total: 13.8s	remaining: 397ms
2916:	learn: 0.1620121	total: 13.8s	remaining:

In [20]:
print("Accuracy:", np.mean(np.array(classifier.predict(X_test)).flatten() == np.array(y_sex_test)))
print("Most important features:")
for i, name in enumerate(most_important_features(classifier.get_feature_importance(), features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.8612862547288777
Most important features:
1. 40kg
2. mudakoff
3. girlmeme
4. modnailru
5. 9o_6o_9o
6. 4ch
7. i_d_t
8. igm
9. academyofman
10. femalemem
